In [1]:
from prog_models.models.battery_electrochem import BatteryElectroChemEOD
from prog_algs import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def future_loading(t, x=None):
        load = 1
        return {"i": load}

In [3]:
dt = 1
# Process noise
Q_vars = {
    'tb': 1,
    'Vo': 0.01,
    'Vsn': 0.01,
    'Vsp': 0.01,
    'qnB': 1,
    'qnS': 1,
    'qpB': 1,
    'qpS': 1
}
# Measurement noise
R_vars = {
    't': 2, 
    'v': 0.02
}

In [4]:
battery = BatteryElectroChemEOD(process_noise= Q_vars,
                                measurement_noise = R_vars, 
                                dt = dt)

In [5]:
start_u = future_loading(0)
start_x = battery.initialize(start_u)
start_y = battery.output(start_x)
times, inputs, states, outputs, event_states = battery.simulate_to_threshold(future_loading, start_y, save_freq = 1)

In [6]:
all_particles = []
n_times = int(np.round(np.random.uniform(len(times)*.25,len(times)*.45,1)))

In [7]:
batt_pf = state_estimators.ParticleFilter(model = battery,  x0 = states[0], num_particles = 250)

c:\users\m77774\documents\research\prog_algs\src\prog_algs\state_estimators\particle_filter.py:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = array(list(x0.values()))


In [8]:
batt_pf.for_fun()

1

In [8]:
batt_pf.estimate(t=times[2], u = inputs[2], z=outputs[2])

In [1]:
from prog_models.models import BatteryCircuit as Battery
from prog_algs import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def future_loading(t, x = None):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2
    else:
        i = 3
    return {'i': i}
batt = Battery()
initial_state = batt.parameters['x0']
start_u = future_loading(0)
start_x = batt.initialize(start_u)
start_y = batt.output(start_x)
times, inputs, states, outputs, event_states = batt.simulate_to_threshold(future_loading, start_y, save_freq = 1)


In [5]:
batt.dx(start_x,start_u)

<bound method BatteryCircuit.dx of <prog_models.models.battery_circuit.BatteryCircuit object at 0x00000199FA382640>>

In [7]:
# Step 2: Demonstrating state estimator
print("\nPerforming State Estimation Step")
# Step 2a: Setup
filt = state_estimators.ParticleFilter(batt, initial_state)


Performing State Estimation Step


In [6]:
example_measurements = {'t': 32.2, 'v': 3.915}
t = 0.1
filt.estimate(t, future_loading(t), example_measurements)

AssertionError: New time must be greater than previous

In [12]:
i=1
filt.estimate(t=times[i],u=inputs[i],z=outputs[i])

TypeError: only size-1 arrays can be converted to Python scalars

AssertionError: New time must be greater than previous